In [1]:
from pyomo.environ import *

In [2]:
# Sets
plants = [1, 2, 3]
dcs = list(range(1, 13))
# Parameters
capacity = {1: 120, 2: 90, 3: 100}  # Capacity of each plant
cost = {1: 8, 2: 7, 3: 9}  # Cost per unit of production
price = 20  # Selling price per unit
profit=[12,13,11]
#scrap_price = 5  # Scrap price per unit
#lost_sale_penalty = 7  # Penalty for lost sales
demand = {1: 20, 2: 30, 3: 40, 4: 35, 5: 15, 6: 19, 7: 25, 8: 20, 9: 18, 10: 12, 11: 30, 12: 40}

transport_cost = {
    (1, 1): 2, (1, 2): 3, (1, 3): 2, (1, 4): 3, (1, 5): 5, (1, 6): 4.4, (1, 7): 3.2, (1, 8): 3,
    (1, 9): 1.5, (1, 10): 3, (1, 11): 6.0, (1, 12): 1.0,
    (2, 1): 1.9, (2, 2): 3, (2, 3): 2.4, (2, 4): 2.4, (2, 5): 2, (2, 6): 4.1, (2, 7): 2.8, (2, 8): 3,
    (2, 9): 1.7, (2, 10): 2, (2, 11): 1.0, (2, 12): 4.5,
    (3, 1): 3.1, (3, 2): 1.8, (3, 3): 2.8, (3, 4): 3.4, (3, 5): 2, (3, 6): 2.9, (3, 7): 2.2, (3, 8): 3.2,
    (3, 9): 2, (3, 10): 2.8, (3, 11): 4.5, (3, 12): 6.0 }

In [3]:
# Model
model = ConcreteModel()
# Decision Variables
model.x = Var(plants, domain=NonNegativeReals)  # Production quantity at each plant
model.y = Var(plants, dcs, domain=NonNegativeReals)  # Shipment from plant to DC
model.s = Var(plants, domain=NonNegativeReals)  # Surplus
model.l = Var(dcs, domain=NonNegativeReals)  # Lost sales
# Objective Function
model.obj = Objective(
    expr=sum(profit[i-1] * model.y[i, j] for i in plants for j in dcs) 
        - sum(transport_cost[i, j] * model.y[i, j] for i in plants for j in dcs)
        - sum(price * model.s[i] for i in plants),
    sense=maximize)
# Constraints
# Demand fulfillment constraint
model.demand_constraint = ConstraintList()
for j in dcs:
    model.demand_constraint.add(sum(model.y[i, j] for i in plants) + model.l[j] == demand[j])

# Production allocation constraint
model.production_constraint = ConstraintList()
for i in plants:
    model.production_constraint.add(sum(model.y[i, j] for j in dcs) + model.s[i] == model.x[i])

# Capacity constraint
model.capacity_constraint = ConstraintList()
for i in plants:
    model.capacity_constraint.add(model.x[i] <= capacity[i])

In [4]:
# Solve the model
solver = SolverFactory('glpk')
solver.solve(model, tee=True)

# Print results
print(f"Objective Value: {model.obj()}\n")
for i in plants:
    print(f"Production at Plant {i}: {model.x[i]()} units")
    print(f"Surplus at Plant {i}: {model.s[i]()} units")
for j in dcs:
    print(f"Lost Sales at DC {j}: {model.l[j]()} units")
for i in plants:
    for j in dcs:
        print(f"Ship {model.y[i, j]()} units from Plant {i} to DC {j}")

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpzcu5xb8o.glpk.raw --wglp /tmp/tmp9ww0xc0c.glpk.glp --cpxlp
 /tmp/tmpcmgvxynz.pyomo.lp
Reading problem data from '/tmp/tmpcmgvxynz.pyomo.lp'...
18 rows, 54 columns, 93 non-zeros
249 lines were read
Writing problem data to '/tmp/tmp9ww0xc0c.glpk.glp'...
222 lines were written
GLPK Simplex Optimizer 5.0
18 rows, 54 columns, 93 non-zeros
Preprocessing...
15 rows, 39 columns, 75 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 15
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (36)
*    21: obj =   3.065200000e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.1 Mb (65679 bytes)
Writing basic solution to '/tmp/tmpzcu5xb8o.glpk.raw'...
81 lines were written
Objective Value: 3065.2

Production at Plant 1: 120.0 units
Surplus at Pl